# Projeto 2 - Classificador Automático de Sentimento

## Giovanna Cabral e Amanda Rufino - 2A

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
dados= pd.read_excel('tweets_samsung_classificados.xlsx')
dados.head(4)

,Treinamento,relevancia,positividade
0,@samsungbrasil j5 prime samsung. meu cartão ai...,0,1
1,minha tia falando que não troca samsung por ap...,1,0
2,rt @beatriz5correia: pls rt para encontrar o m...,0,0
3,@michaelserra aqui só se for com rom customiza...,1,0


In [3]:
index=0
for frase in dados["Treinamento"]:
    if index<=299:
        frase_new=re.split(" ", frase)
        dados["Treinamento"][index]=frase_new
        index+=1

C:\Users\gi_gi\Anaconda3\ciência dos dados\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [4]:
import nltk
nltk.download('stopwords')

index=0
import string 

print(string.punctuation)
#remove ponctuation from each word 

table = str.maketrans('', '', string.punctuation)
for e in dados["Treinamento"]:
    if index<=299:
        sem_pontuacao=[w.translate(table) for w in dados["Treinamento"][index]]
        dados["Treinamento"][index]=sem_pontuacao
        index+=1


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gi_gi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


C:\Users\gi_gi\Anaconda3\ciência dos dados\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
index=0
palavras_neg_list=[]
palavras_pos_list=[]
palavras_irrel_list=[]
palavras_rel_list=[]
palavras_possiveis=[]
    
for e in dados["relevancia"]:
    for i in dados["Treinamento"][index]:
        palavras_possiveis.append(i)
        
    if e==1:  # relevante.
        for i in dados["Treinamento"][index]:
            palavras_rel_list.append(i)
            
        if dados["positividade"][index] == 0:
             for i in dados["Treinamento"][index]:
                palavras_neg_list.append(i)
        else:
             for i in dados["Treinamento"][index]:
                palavras_pos_list.append(i)
    else:
        for i in dados["Treinamento"][index]:
            palavras_irrel_list.append(i)

    index+=1

def fazDicionario(lista):
    dicionario={}
    for p in lista:
        if p not in dicionario:
            dicionario[p]=0
        dicionario[p]+=1
            
    return dicionario
        
palavras_irrel=fazDicionario(palavras_irrel_list)
palavras_rel=fazDicionario(palavras_rel_list)
palavras_neg=fazDicionario(palavras_neg_list)
palavras_pos=fazDicionario(palavras_pos_list)
num_relevante=fazDicionario(dados["relevancia"])
num_positividade=fazDicionario(dados["positividade"])
    
# https:
def remove_repetidos(lista):
    l = []
    for i in lista:
        if i not in l:
            l.append(i)
    l.sort()
    return l

palavras_possiveis=remove_repetidos(palavras_possiveis)
palavras_rel_unicas=remove_repetidos(palavras_rel)

stop_words=nltk.corpus.stopwords.words('portuguese')


In [6]:
data= pd.read_excel('tweets_teste.xlsx')
data.head(4)

FileNotFoundError: [Errno 2] No such file or directory: 'tweets_teste.xlsx'

In [ ]:
index=0
table = str.maketrans('', '', string.punctuation)
for frase in data["Teste"]:
    if index<=199:
        frase_new=re.split(" ", frase)
        data["Teste"][index]=frase_new
       
        sem_pontuacao=[w.translate(table) for w in data["Teste"][index]]
        data["Teste"][index]=sem_pontuacao
        index+=1
            

In [ ]:
import math

prob_irrel=math.log(num_relevante[0]/(num_relevante[0]+num_relevante[1]))
prob_rel=math.log(num_relevante[1]/(num_relevante[0]+num_relevante[1]))

frases_relevantes=[]
frases_irrelevantes=[]

i=0
while i<199:
    multiplicacao_prob_rel=0
    multiplicacao_prob_irrel=0
 
    frase=data["Teste"][i]
    for palavra in frase:
        #probabilidade da frase ser relevante:
        if palavra not in stop_words:
            quantidade_contagem = 1
            quantidade_contagem_ir = 1
            
            if palavra in palavras_rel:
                quantidade_contagem+=palavras_rel[palavra]

            multiplicacao_prob_rel+=math.log((quantidade_contagem/(len(palavras_rel_list)+len(palavras_possiveis)))) #suavizado por laplace 
        #probabilidade da frase ser irrelevante:    
            
            if palavra in palavras_irrel:
                quantidade_contagem_ir+=palavras_irrel[palavra]
            
            multiplicacao_prob_irrel+=math.log((quantidade_contagem_ir/(len(palavras_irrel_list)+len(palavras_possiveis))))

    multiplicacao_prob_irrel+=prob_irrel    
    multiplicacao_prob_rel+=prob_rel
    
    data["positividade_programa"]=0
    if multiplicacao_prob_rel>multiplicacao_prob_irrel:
        frases_relevantes.append(frase)
        frases_relevantes.append(i)
        data["relevante_programa"][i]=1
        
    else:
        frases_irrelevantes.append(frase)
        data["relevante_programa"][i]=0
        
    i+=1
    

print(frases_relevantes[0])

In [ ]:
prob_pos=math.log(num_positividade[1]/(num_positividade[0]+num_positividade[1]))
prob_neg=math.log(num_positividade[0]/(num_positividade[0]+num_positividade[1]))


frases_positivas=[]
frases_negativas=[]

i=0
while i<len(frases_relevantes):
    multiplicacao_prob_pos=0
    multiplicacao_prob_neg=0
 
    frase=frases_relevantes[i]
   
    for palavra in frase:
        #probabilidade da frase ser relevante:
        if palavra not in stop_words:
            quantidade_contagem_pos = 1
            quantidade_contagem_neg = 1
            
            if palavra in palavras_pos:
                quantidade_contagem_pos+=palavras_pos[palavra]

            multiplicacao_prob_pos+=math.log((quantidade_contagem_pos/(len(palavras_pos_list)+len(palavras_rel_unicas)))) #suavizado por laplace 
        #probabilidade da frase ser irrelevante:    
            
            if palavra in palavras_neg:
                quantidade_contagem_neg+=palavras_neg[palavra]
            
            multiplicacao_prob_neg+=math.log((quantidade_contagem_neg/(len(palavras_neg_list)+len(palavras_rel_unicas))))

    multiplicacao_prob_neg+=prob_neg    
    multiplicacao_prob_pos+=prob_pos
    
    
    if multiplicacao_prob_pos>multiplicacao_prob_neg:
        frases_positivas.append(frase)
        data["positividade_programa"][i+1]=1
        
    else:
        frases_negativas.append(frase)
        data["positividade_programa"][i+1]=0
        
    i+=2
    

In [ ]:
data.head(5)

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [ ]:
relevantes_verd=0
relevantes_falsos=0
irrelevantes_falsos=0
irrelevantes_verd=0
i=0
erro=0
while i<199:
    if data["relevante"][i]==1:
        if data["relevante_programa"][i]==0:
            relevantes_falsos+=1
        if data["relevante_programa"][i]==1:
            relevantes_verd+=1
            
    if data['relevante'][i]==0:#irrelevantes --> 98
        if data['relevante_programa'][i]==0:
            irrelevantes_verd+=1
        if data['relevante_programa'][i]==1:
            irrelevantes_falsos+=1
            
    i+=1
    
print(relevantes_verd)
print(relevantes_falsos)

n_relevante=fazDicionario(data["relevante"])

print(relevantes_verd/n_relevante[1]*100) #probabilidade do programa ter colocado como relevante uma frase que nós 
#tinhamos colocado como relevantes também
print(irrelevantes_verd/n_relevante[0]*100) #probabilidade do programa ter colocado como irrelevante
#dadas as frases colocadas por nos como irrelevantes
print(irrelevantes_falsos/n_relevante[0]*100)
print((relevantes_verd+irrelevantes_verd)/(n_relevante[0]+n_relevante[1])*100) #probabilidade de ter havido um acerto dado todo 
#o conjunto de frases classificado

In [ ]:
acerto_verd=0
acerto_falso=0
acerto_verd_s=0
acerto_falso_s=0
i=0
while i<199:
    if data["relevante_programa"][i]==1: 
        ### nessa parte o programa faz a comparação de ter acertado (ter correspondencia entre 
        #oq foi feito por nos e coloca do ele, dentro de todo o grupo de palavras que ele classificoi como relevantes, 
        #sem necessariamente considerar seus próprios erros)
        if data["positividade_programa"][i]==data["positividade"][i]:
             acerto_verd_s+=1
        else:
            acerto_falso_s+=1
        #Já nessa outra parte o programa verifica se houve acerto para as frases que o programa acertou como relevantes,
        #ou seja, nessa parte só entram os relevantes do programa que tmabém foram considerados por nós como relevantes.
        if data["relevante"][i]==1:
            if data["positividade_programa"][i]==data["positividade"][i]:
                acerto_verd+=1
            else:
                acerto_falso+=1
                
    i+=1 
print(acerto_verd)
print(acerto_falso)
print(acerto_verd_s/(len(frases_relevantes)/2)*100)
print(acerto_verd/(n_relevante[0]+n_relevante[1])*100)

        

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Com base nos resultados obtidos pelo classificador pode-se concluir que há uma probabilidade de acerto de 74%. Dada uma certa frase classificada por nós como relevante há 86% de chance do classificador também ter marcado como relevante. Do mesmo, dada uma frase classificada por nós como irrelevante há 61% de chance do classificador ter marcado como irrelevante. Assim, percebemos que há uma maior chance de erro para frases irrelevantes, o que pode ser explicado pela maior quantidade de frases repetidas no banco de dados irrelevantes, o que diminuiu a precisão do classificador.

Além disso, o classificador realiza uma segunda classificação para as frases, caso essas sejam consideradas relevantes. Dado um conjunto de frases recebido por ele, a chance dele acertar a classificação, ou seja, colocar uma resposta correspondente a colocada por nós, é de 70%. No entanto, como já há um erro nessa segunda lista em que ele analisa, visto que o classificador tem uma chance de classificar como relevante sem o ser, o número da combinação é de 30,5%. 

Por fim, a chance de, dado um conjunto de dados, classificar tanto como relevante corretamente, quanto como positivo/negativo corretamente é de 30%. Apesar do numero parecer pequeno, é bastante satisfatório, pois na segunda passagem, há razoáveis erros na lista classificada, que pode ter relevantes, que não o são, quanto pode faltar relevantes que foram por nós considerado, e com base nessa lista com frases que sobram e frases que faltam que é feita a segunda classificação que também tem uma margem de erro. 

Assim, percebe-se que para melhorar a precisão final do classificador, pode ser implementado melhorias no sentido de aumentar a sensibilidade para os relevantes, a fim de que mais frases possam passar pela segunda classificação. No entanto, o programa ainda é muito simples e traz resultados que demonstram que cada etapa apesar de funcionar bem separadamente,ao serem combinadas não tem alta precisão, o que indica a necessidade de que uma pessoa avalie os dados coletados a partir de duas passgens pelo Naive-Baise confeccionado por nós. 

Além disso, as frases que haviam ironia e duplo sentido são classificadas por nós como interpretações do sentido expressado pela frase, mas não necessariamente as palavras exclusivamente ajudem a afirmar a relevância ou não, nem a sua positividade ou não, de modo que esse conteúdo também pode ter atrapalhado no sucesso das correspondências.

O classificador Naive-Bayes pode ser utilizado para diferentes propósitos, além da classificação de tweets, como aplicado nesse projeto. Outra aplicação para esse classificador seria para diagnósticos médicos. Por exemplo, em um diagnóstico médico existem duas classes possíveis: o paciente tem H1N1 e o paciente não tem H1N1. Além disso, as características possíveis são resultados de um exame com duas possíveis classificações, positivo e negativo. Assim, tendo o conhecimento da probabilidade da população ter essa doença e sabendo a probabibilidade do exame retornar um resultado positivo correto e retorna um resultado negativo correto é possível fazer o diagnóstico médico de determinada pessoa, através do classificador Naive-Bayes. 

Além de ser implementado para diagnósticos médicos, o classificador pode ser utilizado para detecção de fraudes, ou seja, para  identificar se uma transação financeira é “legal” ou “suspeita”, para programas de filtragem de spam, ou seja, detectar se e-mail é um spam ou não. Outro propósito possível seria a aprovação de crédito, ou seja, classificar um cliente de acordo com o risco que ele irá trazer para a concessão de crédito. Consequentemente, pode-se concluir que o classificador Naive-Bayes tem várias aplicações com diferentes propósitos e grande utilidade. 


